<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/20_workbook/ProgramWorkBook_CSVFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# プログラム記述練習：CSVファイル編

まずはこのプログラムをコピーして自分のGoogleドライブへと保存しましょう．

## 0. データのダウンロード

プログラムに必要なデータをダウンロードします．

今回使用するデータは，Kaggleで公開されているBitcoinの価格を予測するデータセットです．

https://www.kaggle.com/datasets/team-ai/bitcoin-price-prediction

**今回使用するデータは，CSVファイルなどのテキストファイルで整理・保存されたデータを扱うための練習として採用しました．実際の現場などでは，センサーの数値データなどをCSVファイルに保存しておき，その数値データをもとに学習するような状況を想定しています．**





In [ ]:
!wget -q http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/BitcoinPricePrediction.zip
!unzip -q BitcoinPricePrediction.zip

ここで，一度データセットを確認してみましょう．

データ（フォルダ）を確認すると，BitcoinPricePredictionフォルダの中にTraining.csvとTest.csvという二つのCSVファイルが保存されています．

![BitcoinDataDir.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/e81aa369-427e-fe22-03cf-03cd82c917ce.png)

### CSVファイルの中身

それぞれの中身を見ると，
* Date: 日付
* Open: 始値
* High: 最高値
* Low: 最安値
* Close: 終値
* Volume: 取引ボリューム（取引数量）
* Market Cap: 時価総額
という列があり，それぞれの日付で値を持っていることがわかります．

また，Dateの値を確認すると，新 --> 古の順番に日付が並んでいることがわかります．

今回は，「Open, High, Low, Close」の値から翌日の「High, Low」を予測する再帰型ニューラルネットワークを構築して学習してみましょう．

## 1. モジュールのインポート部分

こちらにプログラムに必要なモジュールのインポートを記述しましょう．

※ プログラムが一通り完成した後に記述します．

## 2. データセットクラスの作成

ここでは，ダウンロードしたCSVファイルの形式に合わせて，PyTorchのデータセットクラスを自作します．

### この部分で使用する関数・クラスの動作確認

In [ ]:
import os
import glob
import torch
from torch.utils.data import Dataset
import csv

### CSVファイルの読み込み
with open("BitcoinPricePrediction/Test.csv", "r") as f:
  reader = csv.reader(f)
  print("print reader:", reader, "\n-----------------")  # ここではcsv.readerというクラスオブジェクトという情報しか表示されない

  # for文で1行ずつリストとして読み込み
  for row in reader:
    print(row)

### リストの並べ替え（逆順）
_sample_list = [1,2,3,4,5]
print("before reverse:", _sample_list)
_sample_list.reverse()
print("after reverse: ", _sample_list)

### Tensor配列の上下・左右反転
_sample_array = torch.arange(6).view(2, 3)
print("sample array:\n", _sample_array)

_updown_array = torch.flipud(_sample_array)
_leftright_array = torch.fliplr(_sample_array)
print("flip up-down array:\n", _updown_array)
print("flip left-right array:\n", _leftright_array)

上で確認した機能を使いつつ，データセットクラスを定義します．

### データセットクラスのデバッグ

ここで，一度作成したデータセットクラスを呼び出して実行することで，正しく動作しているかを確認（デバッグ）します．

**※ 細かなデバッグは非常に重要です．全てのプログラムを記述してからデバッグをしようとすると，原因の特定に時間がかかります．一つのクラス・関数などを作成したら，簡単なプログラムを記述してうまく動作するか確認をすることをお勧めします．**

## 3. ネットワークモデルの定義

続いてネットワークを定義します．

今回は再帰型ニューラルネットワークを定義します．




この時，各レイヤーのクラスの詳細などを調べたい場合は，PyTorchのReferenceを参照しつつ実装します．

[PyTorch reference mannual](https://pytorch.org/docs/stable/index.html)



### ネットワーククラスのデバッグ

ここで，一度作成したネットワーククラスを呼び出して実行することで，正しく動作しているかを確認（デバッグ）します．

## 4. 学習の準備

ここでは，学習に必要な

* ネットワークモデル
* 誤差関数
* 最適化手法
* データセット

の定義を行います．

最適化関数などもReference mannualを参照しつつ好きなものを選択記述しましょう．

**DataLoaderのnum_workersについて**

`torch.utils.data.DataLoader`の引数である`num_workers`は，データを読み込んで準備する処理を並列処理するための引数です．例えば，`num_workers=10`とした場合には，10並列でデータの読込処理 (データセットクラスの`__getitem__()`) を10並列で実行してくれます．そのため，使用する計算機のCPU性能に合わせて，ある程度大きな数を指定しておくとデータの読込処理が早くなり，学習の高速化が期待できます．

 ## 5. 学習の開始

 上で定義したモデルや最適化手法，データセットを用いて学習を行います．

### この部分で使用する関数・クラスの動作確認

In [ ]:
### item (tensorのとある一つの要素をscalerとして返す)
_sample_tensor = torch.tensor([1,2,3,4], dtype=torch.float32)
print("return as scaler:", _sample_tensor[0].item())

### 上のセルで定義したネットワークモデルのパラメータ（+そのほか）の抽出
print(list(model.parameters()))   # 学習するモデルパラメータ（optimizerへ入力する時に使用することが多い）
print("----------------")
print(model.state_dict())         # 学習するパラメータ + 学習はしないけど学習や推論処理を行うことで変化するパラメータ
                                  #  (BatchNormの内部パラメータなど)（モデルの保存などに使用されることが多い）
print("----------------")

### PyTorchのtensorのファイル保存
_save_data = torch.tensor([1,2,3,4], dtype=torch.float32)
torch.save(_save_data, "sample_data.pt")

### PyTorchのtensorのファイルの読込
_load_data = torch.load("sample_data.pt")
print(_load_data)

上記の関数を活用して学習ループを記述します．

### 学習経過のテキスト保存とグラフ化

**テキスト保存**

ログ（コンソール・ターミナル部分）に学習中の誤差の推移などを表示することで，学習の様子を確認できますが，Colaboratoryの再起動や再度学習を行うことでログは消えてしまい．後程，学習経過を確認・比較することができなくなります．
そのため，学習経過の数値をテキストファイルなどで保存しておくと，非常に便利です．
異なるネットワークや学習パラメータを用いて学習を行った場合との比較などを行うことができます．



**グラフ化**

実行結果の数値を見ることで学習の様子が確認できますが，グラフ化することでより直感的に学習の挙動を理解することができます．
ここでは，上の学習プログラム中で保存しておいた，1 epochごとの誤差とテストデータに対する精度をグラフ化して確認してみます．

## 6. 評価

学習したモデルを用いて評価を行います．

### モデルの読み込み

学習中に保存したモデルを読み込みます．

### 推論と結果の表示

読み込んだ学習済みモデルを用いて予測を行います．

今回のネットワークは，数値データ（価格）の予測値を出力するモデルのため，誤差ではなく，その予測値をリストに保存して，後程グラフ表示をして結果を確認します．


**学習データに対する予測**

まずは，学習に使用したデータでどの程度予測できるかを確認します．

**テストデータに対する予測**

次に，テスト用データでどの程度予測できるかを確認します．